In [ ]:
# Import necessary libraries
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import subprocess
from IPython.display import display, HTML, Video
from tqdm.notebook import tqdm
import shutil
import tempfile

In [ ]:
# Check if ffmpeg is installed and accessible
def check_ffmpeg():
    """Check if ffmpeg is installed and accessible."""
    try:
        result = subprocess.run(['ffmpeg', '-version'], 
                               stdout=subprocess.PIPE, 
                               stderr=subprocess.PIPE)
        if result.returncode == 0:
            ffmpeg_version = result.stdout.decode().split('\n')[0]
            print(f"FFmpeg is installed: {ffmpeg_version}")
            return True
        else:
            print("FFmpeg is not accessible.")
            return False
    except FileNotFoundError:
        print("FFmpeg is not installed or not in PATH.")
        print("Please install FFmpeg from https://ffmpeg.org/download.html")
        return False

# Check for FFmpeg before proceeding
if not check_ffmpeg():
    raise RuntimeError("FFmpeg is required for video creation.")

In [ ]:
# Set path to the folder containing drone images
image_folder = "S:/Savanna Institute/Imagery/Hazelnut/Biomass Estimation/Rosemount Hazelnut Biomass/ArcGIS_Pro_Hazelnut_Biomass_Estimation/original_data/Raw Images/"

# Create a list of image files and sort them
# Support multiple image extensions
image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.tif', '*.tiff']
image_files = []

for ext in image_extensions:
    image_files.extend(glob.glob(os.path.join(image_folder, ext)))
    
# Sort the files alphanumerically
image_files = sorted(image_files)

# Display the total number of images found
print(f"Found {len(image_files)} images in the directory.")

max_images = 300
if len(image_files) > max_images:
    image_files = image_files[:max_images]
    print(f"Limiting processing to first {max_images} images")

# Display the first few file names
print("Sample file names:")
for file in image_files[:5]:
    print(f"- {os.path.basename(file)}")

In [ ]:
# Function to display image properties
def display_image_info(image_path):
    with Image.open(image_path) as img:
        width, height = img.size
        mode = img.mode
        
        # Get image properties
        size_kb = os.path.getsize(image_path) / 1024
        
        # Display information
        print(f"File: {os.path.basename(image_path)}")
        print(f"Dimensions: {width}x{height} pixels")
        print(f"Mode: {mode}")
        print(f"File size: {size_kb:.2f} KB")
        
        # Display the image
        plt.figure(figsize=(10, 8))
        plt.imshow(img)
        plt.axis('off')
        plt.show()

# Preview the first 3 images (if available)
num_preview = min(3, len(image_files))
for i in range(num_preview):
    display_image_info(image_files[i])

In [ ]:
# Create a temporary directory for processed images
temp_dir = tempfile.mkdtemp()
print(f"Created temporary directory: {temp_dir}")

# Function to process images with optimized settings
def process_image(source_path, target_path, resize=None):
    """Process an image and save to target path"""
    with Image.open(source_path) as img:
        if resize:
            img = img.resize(resize, Image.LANCZOS)
        
        # Save the processed image (use JPG for better compression & performance)
        img.save(target_path, "JPEG", quality=90)
        return img.size

# Determine optimal resolution (resize if images are too large)
with Image.open(image_files[0]) as img:
    width, height = img.size
    
    # Determine if resizing is needed (for smooth video playback)
    max_dimension = 1920  # Full HD resolution
    resize_dimensions = None
    
    if width > max_dimension or height > max_dimension:
        # Calculate new dimensions while maintaining aspect ratio
        if width > height:
            new_width = max_dimension
            new_height = int(height * (max_dimension / width))
        else:
            new_height = max_dimension
            new_width = int(width * (max_dimension / height))
        
        resize_dimensions = (new_width, new_height)
        print(f"Resizing images from {width}x{height} to {new_width}x{new_height} for smoother playback")
    else:
        print(f"Using original resolution: {width}x{height}")

# Process images
print("Processing images...")
processed_paths = []
try:
    for i, img_path in enumerate(tqdm(image_files)):
        # Create numbered output filename (use jpg for better performance)
        output_filename = f"frame_{i:06d}.jpg"
        output_path = os.path.join(temp_dir, output_filename)
        
        # Process and save
        process_image(img_path, output_path, resize=resize_dimensions)
        processed_paths.append(output_path)
    
    print(f"Processed {len(processed_paths)} images")
    
    # Set output path for video directly in the image folder
    output_video_path = os.path.join(image_folder, "drone_footage_10.mp4")
    
    # Lower frame rate for smoother playback
    fps = 10
    
    # Define ffmpeg command with optimized settings for smoother playback
    ffmpeg_cmd = [
        'ffmpeg',
        '-y',  # Overwrite output file if it exists
        '-framerate', str(fps),
        '-i', os.path.join(temp_dir, 'frame_%06d.jpg'),
        '-c:v', 'libx264',
        '-crf', '23',  # Slightly lower quality but better compression
        '-preset', 'medium',  # Better balance of speed vs compression
        '-tune', 'film',  # Better for natural imagery
        '-pix_fmt', 'yuv420p',  # For compatibility
        '-movflags', '+faststart',  # Optimize for streaming/playback
        output_video_path
    ]
    
    # Run ffmpeg command
    print("Creating video with ffmpeg...")
    print(f"Running command: {' '.join(ffmpeg_cmd)}")
    
    result = subprocess.run(ffmpeg_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    
    if result.returncode == 0:
        print(f"Video created successfully at: {output_video_path}")
    else:
        print("Error creating video:")
        print(result.stderr.decode())
        
except Exception as e:
    print(f"Error: {str(e)}")
finally:
    # Clean up temporary directory
    print(f"Cleaning up temporary directory: {temp_dir}")
    shutil.rmtree(temp_dir)

In [ ]:
# Display video information and preview
if os.path.exists(output_video_path):
    video_size_mb = os.path.getsize(output_video_path) / (1024 * 1024)
    video_duration = len(image_files) / fps
    
    print(f"Video file size: {video_size_mb:.2f} MB")
    print(f"Video duration: {video_duration:.2f} seconds")
    print(f"Frame rate: {fps} fps")
    print(f"Resolution: {resize_dimensions if resize_dimensions else (width, height)}")
    
    # Display the video in the notebook
    display(Video(output_video_path, embed=True))
    
    # Print instructions for accessing the video file
    print(f"\nVideo is saved at: {output_video_path}")
else:
    print("Video file was not created successfully.")